In [2]:
!pip install qutip

     |████████████████████████████████| 14.6 MB 2.1 MB/s 


In [80]:
#Import libraries 
import qutip as qt
import numpy as np
from numpy import linalg
import itertools
import matplotlib.pyplot as plt
from qutip.qip.operations import swap
import pickle
import json 
import os


In [59]:
#System variables

N= 10 # Majorana fermions number
#f= fermion_operators(N//2+1) # Complex fermions anihilation operators
w= 1e-1
g= 1e-1
gamma= 1e-1
beta = 1e-2
times = np.linspace(0.0, 6.0, 100)


class OpenSYK():
  def __init__(self,w_g, N, gamma_beta, model):
    # Model
    self.Model= model
   
    # Central fermion strength 
    self.w= 1e-1
    self.w_g= w_g

    # Interaction strength
    self.g = (1/self.w_g)* self.w

    # Majorana fermions number
    self.N= N

    # Coupling function strength 
    self.gamma= 1e-1
    self.gamma_beta = gamma_beta
    # Bath temperature
    self.beta = self.gamma*(1/self.gamma_beta)

    #number of fermions = [complex fermions + central fermion(model 1), complex fermions(model 2)]
    self.n1 = N//2+1
    self.n2 = N//2

    #model
   # self.model= model
  
  #Define fermion operators using the Jordan-Wigner transform (n or N//2)
  @property
  def fermion_operators(self):
    if self.Model==1:
      return [qt.tensor(*[qt.destroy(2) if i == j\
                  else (qt.sigmaz() if j < i\
                      else qt.identity(2))\
                          for j in range(self.n1)])\
                              for i in range(self.n1)]
    if self.Model==2:
      return [qt.tensor(*[qt.destroy(2) if i == j\
                  else (qt.sigmaz() if j < i\
                      else qt.identity(2))\
                          for j in range(self.n2)])\
                              for i in range(self.n2)]

  
  #Hamiltonian of the system SYK+central fermion
  @property
  def Hamiltonian1(self):
    c_fermi = self.fermion_operators
    wg = np.zeros(self.n1)
    wg[0] = self.w/2 # isolated fermion energy
    wg[1:]= self.g   # interaction strength

    #random interaction matrix 
    K = np.random.randn(self.N,self.N) 
    K = 0.5*(K - K.T)

    # K_ab eigenenergies
    lbd= np.linalg.eigvals(K)
    lbd=[np.imag(lbd)[x] for x in range(0,len(lbd),2)]

    #define M
    M=np.zeros((self.n1)**2)
    M= M.reshape(self.n1,self.n1)
    M[0]= wg
    for x in range(self.n2): 
      M[x+1][x+1] =lbd[x]/2
    M = M + M.T
    epsilon = np.linalg.eigvals(M)

    #coupling constants
    norms= np.sqrt([1+g**2*sum(1/(lbd[i]-epsilon[j])**2 for i in range(len(lbd))) for j in range(len(epsilon))])
    random_c = np.array([1+g*sum(1/(lbd[i]-epsilon[j]) for i in range(len(lbd))) for j in range(len(epsilon))])
    random_c = (random_c/norms)**2
    random_cps = [N*gamma*(1+np.tanh(0.5*beta*epsilon[i]))*random_c[i] for i in range(len(random_c))]

    # diagonalized Hamiltonian
    H_d = sum([ epsilon[i]*(c_fermi[i].dag()*c_fermi[i]) for i in range(self.n2)]) 
    return  H_d, epsilon, lbd, random_cps




In [32]:

def solve1(coupl_c,times,c_fermi,N,H):

  #jump operators
  J_op = [np.sqrt(coupl_c[i])*c_fermi[i] for i in range(N//2+1)]
  J_opdag=[J_op[j].dag() for j in range(len(J_op))]
  J_ops = J_op.append(J_opdag)

  #collapse operators
  C_op= [J_op[i].dag()*J_op[i] for i in  range(N//2+1)]

  #initial state
  psi_0 =qt.tensor(*[qt.basis(2,1) for j in range(N//2+1)])
  

  #evolve and calculate expectation values for each quasi-fermion
  out= qt.mesolve(H, psi_0, times, J_op, C_op)
  rho= qt.mesolve(H, psi_0, times, J_op, [])
  return out, rho

In [78]:
#Statistical sampling for the system dynamics 
# N=14 t= 1.21 minutes per iteration
# N=12 t= 17s per iteration
# N=10 t= 3s per iteration
parameters= {"w/g":np.linspace(1, 100, 10), "N":[10, 12, 14], "gamma/beta" :np.linspace(.01,100,10), "AverageNumber":50}

cases=parameters["w/g"].size

iterations= parameters["AverageNumber"]

Models = [OpenSYK(parameters["w/g"][i], parameters["N"][1],parameters["gamma/beta"][i], 1 )for i in range(cases)]



In [79]:
times = np.linspace(0.0, 6, 100)

##################################################
#Calculate the time evolution of the system

##################################################
for i in range(cases):
 purity=[]
 entropy=[]
 data= {"counts":[], "purity":[], "entropy":[]}
 for j in range(iterations):
    H, eps, lamb, coupl_c = Models[i].Hamiltonian1
    result, rho = solve1(coupl_c, times,  Models[i].fermion_operators, Models[i].N, H )
    counts= result.expect
    rhot = rho.states
    purity_min = [1/2**(N//2+1)]*len(rhot)
    entropy_max= [np.log(2**(N//2+1))]*len(rhot)
    for k in range(len(rhot)):
      # purity
      purity.append((rhot[k]**2).tr())
      #entropy
      entropy.append(qt.entropy_vn(rhot[k]))
    data["counts"].append(counts)
    data["purity"].append(purity)
    data["entropy"].append(entropy)   
 with open('w_g'+'%.3f_'%(Models[i].w_g)+'N%.1f_'%(parameters["N"][1])+'gamma_beta%.2f'%(Models[i].gamma_beta)+'.pickle','wb') as handle:
      pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
 print(i+"batches")




FileNotFoundError: ignored